<a href="https://colab.research.google.com/github/yinpu/Recommendation-algorithm/blob/main/SASRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 数据处理函数

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from collections import defaultdict
from tensorflow.keras.preprocessing.sequence import pad_sequences


def sparseFeature(feat, feat_num, embed_dim=4):
    """
    create dictionary for sparse feature
    :param feat: feature name
    :param feat_num: the total number of sparse features that do not repeat
    :param embed_dim: embedding dimension
    :return:
    """
    return {'feat': feat, 'feat_num': feat_num, 'embed_dim': embed_dim}


def create_ml_1m_dataset(file, trans_score=2, embed_dim=8, maxlen=40, test_neg_num=100):
    """
    :param file: A string. dataset path.
    :param trans_score: A scalar. Greater than it is 1, and less than it is 0.
    :param embed_dim: A scalar. latent factor.
    :param maxlen: A scalar. maxlen.
    :param test_neg_num: A scalar. The number of test negative samples
    :return: user_num, item_num, train_df, test_df
    """
    print('==========Data Preprocess Start=============')
    data_df = pd.read_csv(file, sep="::", engine='python',
                          names=['user_id', 'item_id', 'label', 'Timestamp'])
    # filtering
    data_df['item_count'] = data_df.groupby('item_id')['item_id'].transform('count')
    data_df = data_df[data_df.item_count >= 5]
    # trans score
    data_df = data_df[data_df.label >= trans_score]
    # sort
    data_df = data_df.sort_values(by=['user_id', 'Timestamp'])
    # split dataset and negative sampling
    print('============Negative Sampling===============')
    train_data, val_data, test_data = defaultdict(list), defaultdict(list), defaultdict(list)
    item_id_max = data_df['item_id'].max()
    for user_id, df in tqdm(data_df[['user_id', 'item_id']].groupby('user_id')):
        pos_list = df['item_id'].tolist()

        def gen_neg():
            neg = pos_list[0]
            while neg in set(pos_list):
                neg = random.randint(1, item_id_max)
                return neg

        neg_list = [gen_neg() for i in range(len(pos_list) + test_neg_num)]
        for i in range(1, len(pos_list)):
            hist_i = pos_list[:i]
            if i == len(pos_list) - 1:
                test_data['hist'].append(hist_i)
                test_data['pos_id'].append(pos_list[i])
                test_data['neg_id'].append(neg_list[i:])
            elif i == len(pos_list) - 2:
                val_data['hist'].append(hist_i)
                val_data['pos_id'].append(pos_list[i])
                val_data['neg_id'].append(neg_list[i])
            else:
                train_data['hist'].append(hist_i)
                train_data['pos_id'].append(pos_list[i])
                train_data['neg_id'].append(neg_list[i])
    # item feature columns
    user_num, item_num = data_df['user_id'].max() + 1, data_df['item_id'].max() + 1
    item_feat_col = sparseFeature('item_id', item_num, embed_dim)
    # shuffle
    random.shuffle(train_data)
    random.shuffle(val_data)
    # padding
    print('==================Padding===================')
    train = [pad_sequences(train_data['hist'], maxlen=maxlen), np.array(train_data['pos_id']),
               np.array(train_data['neg_id'])]
    val = [pad_sequences(val_data['hist'], maxlen=maxlen), np.array(val_data['pos_id']),
             np.array(val_data['neg_id'])]
    test = [pad_sequences(test_data['hist'], maxlen=maxlen), np.array(test_data['pos_id']),
             np.array(test_data['neg_id'])]
    print('============Data Preprocess End=============')
    return item_feat_col, train, val, test

In [2]:
file = '/content/drive/MyDrive/RecSysLab/SASRec/ml-1m/ratings.dat'
trans_score = 1
maxlen = 200
test_neg_num = 100
embed_dim = 50

item_fea_col, train, val, test = create_ml_1m_dataset(file, trans_score, embed_dim, maxlen, test_neg_num)

==========Data Preprocess Start=============


  1%|          | 35/6040 [00:00<00:17, 343.54it/s]

============Negative Sampling===============


100%|██████████| 6040/6040 [00:29<00:00, 202.58it/s]


==================Padding===================
============Data Preprocess End=============


In [3]:
train[0]

array([[   0,    0,    0, ...,    0,    0, 3186],
       [   0,    0,    0, ...,    0, 3186, 1270],
       [   0,    0,    0, ..., 3186, 1270, 1721],
       ...,
       [3449, 2301, 1127, ...,  457, 3671,  232],
       [2301, 1127,  592, ..., 3671,  232, 2917],
       [1127,  592, 2407, ...,  232, 2917, 1921]], dtype=int32)

## 模型构建



![image-20210411152238353](https://gitee.com/hello_yinpu/picture/raw/master/imgs/image-20210411152238353.png)

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Embedding, Conv1D

In [56]:
class FFN(Layer):
    def __init__(self, hidden_unit, d_model):
        """
        Feed Forward Network
        :param hidden_unit: A scalar. W1
        :param d_model: A scalar. W2
        """
        super(FFN, self).__init__()
        self.conv1 = Conv1D(filters=hidden_unit, kernel_size=1, activation='relu', use_bias=True)
        self.conv2 = Conv1D(filters=d_model, kernel_size=1, activation=None, use_bias=True)

    def call(self, inputs):
        x = self.conv1(inputs)
        output = self.conv2(x)
        return output


class EncoderLayer(Layer):
    def __init__(self, d_model, num_heads=1, ffn_hidden_unit=128, dropout=0., norm_training=True, causality=True):
        """
        Encoder Layer
        :param d_model: A scalar. The self-attention hidden size.
        :param num_heads: A scalar. Number of heads.
        :param ffn_hidden_unit: A scalar. Number of hidden unit in FFN
        :param dropout: A scalar. Number of dropout.
        :param norm_training: Boolean. If True, using layer normalization, default True
        :param causality: Boolean. If True, using causality, default True
        """
        super(EncoderLayer, self).__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads, d_model)
        self.ffn = FFN(ffn_hidden_unit, d_model)

        self.layernorm1 = LayerNormalization(epsilon=1e-6, trainable=norm_training)
        self.layernorm2 = LayerNormalization(epsilon=1e-6, trainable=norm_training)

        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)

    def call(self, inputs):
        x, mask = inputs

        attention_mask = tf.py_function(generate_attention_mask, [mask], tf.bool)
        attention_mask = tf.reshape(attention_mask, (-1,mask.shape[1], mask.shape[1]))
        att_out = self.mha(x, x, attention_mask=attention_mask)  # （None, seq_len, d_model)

        att_out = self.dropout1(att_out)
        # residual add
        out1 = self.layernorm1(x + att_out)
        # ffn
        ffn_out = self.ffn(out1)
        ffn_out = self.dropout2(ffn_out)
        # residual add
        out2 = self.layernorm2(out1 + ffn_out)  # (None, seq_len, d_model)
        return out2

In [57]:
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Embedding, Input


class SASRec(tf.keras.Model):
    def __init__(self, item_fea_col, blocks=1, num_heads=1, ffn_hidden_unit=128,
                 dropout=0., maxlen=40, norm_training=True, causality=False, embed_reg=1e-6):
        """
        SASRec model
        :param item_fea_col: A dict contains 'feat_name', 'feat_num' and 'embed_dim'.
        :param blocks: A scalar. The Number of blocks.
        :param num_heads: A scalar. Number of heads.
        :param ffn_hidden_unit: A scalar. Number of hidden unit in FFN
        :param dropout: A scalar. Number of dropout.
        :param maxlen: A scalar. Number of length of sequence
        :param norm_training: Boolean. If True, using layer normalization, default True
        :param causality: Boolean. If True, using causality, default True
        :param embed_reg: A scalar. The regularizer of embedding
        """
        super(SASRec, self).__init__()
        # sequence length
        self.maxlen = maxlen
        # item feature columns
        self.item_fea_col = item_fea_col
        # embed_dim
        self.embed_dim = self.item_fea_col['embed_dim']
        # d_model must be the same as embedding_dim, because of residual connection
        self.d_model = self.embed_dim
        # item embedding
        self.item_embedding = Embedding(input_dim=self.item_fea_col['feat_num'],
                         input_length=1,
                         output_dim=self.item_fea_col['embed_dim'],
                         mask_zero=True,
                         embeddings_initializer='random_uniform',
                         embeddings_regularizer=l2(embed_reg))
        self.pos_embedding = Embedding(input_dim=self.maxlen,
                         input_length=1,
                         output_dim=self.embed_dim,
                         mask_zero=False,
                         embeddings_initializer='random_uniform',
                         embeddings_regularizer=l2(embed_reg))
        self.dropout = Dropout(dropout)
        # attention block
        self.encoder_layer = [EncoderLayer(self.d_model, num_heads, ffn_hidden_unit,
                           dropout, norm_training, causality) for b in range(blocks)]

    def call(self, inputs, training=None):
        # inputs
        seq_inputs, pos_inputs, neg_inputs = inputs  # (None, maxlen), (None, 1), (None, 1)
        # mask
        mask = tf.expand_dims(tf.cast(tf.not_equal(seq_inputs, 0), dtype=tf.float32), axis=-1)  # (None, maxlen, 1)
        # seq info
        seq_embed = self.item_embedding(seq_inputs)  # (None, maxlen, dim)
        # pos encoding
        # pos_encoding = positional_encoding(seq_inputs, self.embed_dim)
        pos_encoding = tf.expand_dims(self.pos_embedding(tf.range(self.maxlen)), axis=0)
        seq_embed += pos_encoding
        seq_embed = self.dropout(seq_embed)
        att_outputs = seq_embed  # (None, maxlen, dim)
        att_outputs *= mask

        # self-attention
        for block in self.encoder_layer:
            att_outputs = block([att_outputs, mask])  # (None, seq_len, dim)
            att_outputs *= mask

        # user_info = tf.reduce_mean(att_outputs, axis=1)  # (None, dim)
        user_info = tf.expand_dims(att_outputs[:, -1], axis=1)  # (None, 1, dim)
        # item info
        pos_info = self.item_embedding(pos_inputs)  # (None, 1, dim)
        neg_info = self.item_embedding(neg_inputs)  # (None, 1/100, dim)
        pos_logits = tf.reduce_sum(user_info * pos_info, axis=-1)  # (None, 1)
        neg_logits = tf.reduce_sum(user_info * neg_info, axis=-1)  # (None, 1)
        # loss
        losses = tf.reduce_mean(- tf.math.log(tf.nn.sigmoid(pos_logits)) -
                                tf.math.log(1 - tf.nn.sigmoid(neg_logits))) / 2
        self.add_loss(losses)
        logits = tf.concat([pos_logits, neg_logits], axis=-1)
        return logits

    def summary(self):
        seq_inputs = Input(shape=(self.maxlen,), dtype=tf.int32)
        pos_inputs = Input(shape=(1,), dtype=tf.int32)
        neg_inputs = Input(shape=(1,), dtype=tf.int32)
        tf.keras.Model(inputs=[seq_inputs, pos_inputs, neg_inputs],
                       outputs=self.call([seq_inputs, pos_inputs, neg_inputs])).summary()

In [58]:

trans_score = 1
maxlen = 200
test_neg_num = 100

embed_dim = 50
blocks = 2
num_heads = 1
ffn_hidden_unit = 64
dropout = 0.2
norm_training = True
causality = False
embed_reg = 0  # 1e-6
K = 10

learning_rate = 0.001
epochs = 50
batch_size = 512
model = SASRec(item_fea_col, blocks, num_heads, ffn_hidden_unit, dropout,
        maxlen, norm_training, causality, embed_reg)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        multiple             197650      input_13[0][0]                   
                                                                 input_14[0][0]                   
                                                                 input_15[0][0]                   
__________________________________________________________________________________________________
tf.math.not_equal_4 (TFOpLambda (None, 200)          0           input_13[0][0]                   
____________________________________________________________________________________________

In [59]:
import os
import tensorflow as tf
from time import time
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=learning_rate))

In [60]:
def evaluate_model(model, test, K):
    """
    evaluate model
    :param model: model
    :param test: test set
    :param K: top K
    :return: hit rate, ndcg
    """
    pred_y = - model.predict(test)
    rank = pred_y.argsort().argsort()[:, 0]
    hr, ndcg = 0.0, 0.0
    for r in rank:
        if r < K:
            hr += 1
            ndcg += 1 / np.log2(r + 2)
    return hr / len(rank), ndcg / len(rank)

In [ ]:
results = []
for epoch in range(1, epochs + 1):
    # ===========================Fit==============================
    t1 = time()
    model.fit(
        train,
        validation_data=(val, None),
        epochs=1,
        batch_size=batch_size,
    )
    # ===========================Test==============================
    t2 = time()
    if epoch % 5 == 0:
        hit_rate, ndcg = evaluate_model(model, test, K)
        print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, NDCG = %.4f, '
              % (epoch, t2 - t1, time() - t2, hit_rate, ndcg))
        results.append([epoch + 1, t2 - t1, time() - t2, hit_rate, ndcg])
# ============================Write============================
pd.DataFrame(results, columns=['Iteration', 'fit_time', 'evaluate_time', 'hit_rate', 'ndcg']).\
    to_csv('log/SASRec_log_maxlen_{}_dim_{}_blocks_{}_heads_{}_K_{}_.csv'.
            format(maxlen, embed_dim, blocks, num_heads, K), index=False)

  56/1917 [..............................] - ETA: 2:00:42 - loss: 0.6746

In [41]:
def generate_attention_mask(mask):
  # mask: (B,maxlen,1)
  mask_ = mask.numpy()
  attention_mask_list = np.array([np.eye(mask_.shape[1]) for _ in range(mask_.shape[0])])
  for i in range(mask_.shape[0]):
    mask_i = mask_[i,:,0] #(maxlen,)
    first_true_pos = int(mask_i.shape[0]-np.sum(mask_i))
    for j in range(mask_i.shape[0]) :
      if j > first_true_pos :
        attention_mask_list[i, j, first_true_pos:j] = 1
  return attention_mask_list

In [42]:
mask1 = [[False,False,True],[False,True,True],[True,True,True]]
mask1 = tf.reshape(mask1, shape=(-1,3,1))
mask1

<tf.Tensor: shape=(3, 3, 1), dtype=bool, numpy=
array([[[False],
        [False],
        [ True]],

       [[False],
        [ True],
        [ True]],

       [[ True],
        [ True],
        [ True]]])>

In [43]:
generate_attention_mask(mask1)

array([[[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]],

       [[1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 1.]],

       [[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]]])

In [ ]:

def add_sub(x, y):
    '''
    在此函数中使用纯python编程方式
    '''
    x_ = x.numpy()
    y_ = y.numpy()
    result1 = x_ + y_ - (x_ - y_)
    result2 = x_ + y_ + (x_ - y_)  
    # 返回的就是普通的python对象，但是它会自动转化成tensor来作为最终的结果，是自动完成的
    return result1,result2
 
x = tf.constant([10,20,30])
y = tf.constant([100,200,300])
 
y1,y2 = tf.py_function(func=add_sub, inp=[x, y], Tout=[tf.int32,tf.int32])
print(y1)  
print(y2) 

tf.Tensor([200 400 600], shape=(3,), dtype=int32)
tf.Tensor([20 40 60], shape=(3,), dtype=int32)


In [ ]:
def stit(x):
    '''
    在此函数中使用纯python编程方式
    '''
    x_ = x.numpy()
    x_[x_<0.1] = 0
    x = np.concatenate((x,x),axis=1)
    return x

In [ ]:
def test_model(input_shape):
  input_x = tf.keras.layers.Input(shape=(input_shape,))
  tmp = tf.py_function(func=stit, inp=[input_x], Tout=tf.float32)
  tmp = tf.reshape(tmp,(-1,7))
  return tf.keras.models.Model(inputs=input_x, outputs=tmp)

In [ ]:
t_model = test_model(10)

In [ ]:
t_model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, 10)]              0         
_________________________________________________________________
tf.py_function_22 (TFOpLambd None                      0         
_________________________________________________________________
tf.reshape_15 (TFOpLambda)   (None, 7)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x=tf.constant([[0.1,-0.2,0.3,-10000000,0.5,0.6,0.7,0.8,-1000,0]])

In [ ]:
x.shape

TensorShape([1, 10])

In [ ]:
t_model(x)

InvalidArgumentError: ignored